In [1]:
!pip3 install scikit-learn pandas numpy matplotlib seaborn fairlearn ucimlrepo shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 4.7 MB/s eta 0:00:00


In [4]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [8]:
!pip install spark-nlp
!pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.3/636.3 kB 9.5 MB/s eta 0:00:00


In [10]:
import numpy as np
import torch
import torchvision
import torchvision.transforms.v2 as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
import shap

import random
import pandas as pd
from ucimlrepo import fetch_ucirepo
from typing import Dict, List, Tuple

import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
from fairlearn.adversarial import AdversarialFairnessClassifier

In [18]:
import sparknlp
from sparknlp.base import DocumentAssembler, LightPipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    YakeKeywordExtraction
)
import pyspark.sql.functions as F

In [33]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    YakeKeywordExtraction
)
import pyspark.sql.functions as F


spark = sparknlp.start()


# Step 1: Transforms raw texts to `document` annotation
document = DocumentAssembler().setInputCol("text").setOutputCol("document")

# Step 2: Sentence Detection
sentenceDetector = SentenceDetector().setInputCols("document").setOutputCol("sentence")

# Step 3: Tokenization
token = Tokenizer().setInputCols("sentence").setOutputCol("token").setContextChars(["(", ")", "?", "!", ".", ","])

# Step 4: Keyword Extraction
keywords = YakeKeywordExtraction().setInputCols("token").setOutputCol("keywords")

# Define the pipeline
yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

# Create an empty dataframe
empty_df = spark.createDataFrame([['']]).toDF("text")

# Fit the dataframe to get the
yake_Model = yake_pipeline.fit(empty_df)

light_model = LightPipeline(yake_Model)

df = pd.read_csv('amazon.csv')
df = df.dropna()
df = df.drop_duplicates()

# Filter for product ID, name, review_title, review_content
df_new = df[['product_id', 'product_name', 'review_title', 'review_content']]


# Get the first product ID
first_product_id = df_new['product_id'].iloc[10]

# Filter reviews for the first product
first_product_reviews = df_new[df_new['product_id'] == first_product_id]

# Combine review titles and content into a single string
string = " ".join(first_product_reviews['review_title'] + " " + first_product_reviews['review_content'])

light_result = light_model.fullAnnotate(string)[0]

keys_df = pd.DataFrame([(k.result, k.begin, k.end, k.metadata['score'],  k.metadata['sentence']) for k in light_result['keywords']],
                       columns = ['keywords','begin','end','score','sentence'])
keys_df['score'] = keys_df['score'].astype(float)

# ordered by relevance
keys_df.sort_values(['sentence','score']).head(100)

,keywords,begin,end,score,sentence
0,fast charge,9,19,0.452502,0
1,cable compares,52,65,0.537669,0
78,good for fast,0,12,0.548838,0
6,fast charging,250,262,0.091979,1
5,fast charger,211,222,0.170525,1
...,...,...,...,...,...
82,fast charging stops,435,453,0.920841,6
17,cable design,511,522,0.553154,7
16,probbly something,478,494,1.048331,7
18,cable gets,529,538,0.554847,8


In [32]:
from textblob import TextBlob  # For simple sentiment analysis

# Perform sentiment analysis and calculate score
def calculate_sentiment_score(keys_df):
    total_score = 0
    for _, row in keys_df.iterrows():
        keyword = row['keywords']
        relevance = 1 / (row['score'] + 0.1)  # Invert score, add small constant to avoid division by zero
        sentiment = TextBlob(keyword).sentiment.polarity
        keyword_score = sentiment * relevance
        total_score += keyword_score
    return total_score

# Calculate the sentiment score
sentiment_score = calculate_sentiment_score(keys_df)

# Set a threshold for classification
threshold = 20  # Adjust this based on your data and requirements

# Classify the product
if sentiment_score > threshold:
    classification = "Good"
else:
    classification = "Not Good"

print(f"Sentiment Score: {sentiment_score}")
print(f"Product Classification: {classification}")


Sentiment Score: 2.3339185830990625
Product Classification: Not Good
